# 数据结构和算法

## 1.1. 解压序列赋值给多个变量

Q: 现有一个包含N个元素的元组或序列，怎样将它里面的值解压后同时赋值给N个变量

A：任何的序列（或者可迭代对象）可以通过一个简单的赋值语句解压并赋值给多个变量，唯一的前提及时变量的数量必须跟序列元素的数量是一样的

In [1]:
p = (4, 5)
x, y = p

In [2]:
x

4

In [3]:
y

5

In [4]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
name, shares, price, date = data

In [5]:
name

'ACME'

In [6]:
date

(2012, 12, 21)

有时候只想使用一部分值，可利用任意变量名去占位，然后丢弃，我们常利用'_'

In [8]:
_, shares, price, _ = data
shares

50

## 1.2. 解压可迭代对象赋值给多个变量

Q: 如果一个可迭代对象的元素个数超过变量个数时，会抛出一个 ValueError 。 那么怎样才能从这个可迭代对象中解压出 N 个元素出来？

A: Python 的星号表达式可以用来解决这个问题.使用星号（*）可以接收任意个值

In [9]:
def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)

另外一种情况，假设你现在有一些用户的记录列表，每条记录包含一个名字、邮件，接着就是不确定数量的电话号码。 你可以像下面这样分解这些记录：

In [10]:
record = ('Dave', 'dave@example.com', '773-555-121', '847-968-5555')
name, email, *phone_numbers = record

有一点需要明确的是，得到的phone_number永远是列表类型

值得注意的是，星号表达式在迭代元素为可变长元组的序列时是很有用的。 比如，下面是一个带有标签的元组序列：

In [2]:
records = [
    ('foo', 1, 2),
    ('bar', 'hello'),
    ('foo', 3, 4),
]

def do_foo(x, y):
    print('foo', x, y)

def do_bar(s):
    print('bar', s)

for tag, *args in records:  # 这里遍历records中的每个元组
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


星号解压语法在字符串操作中的应用

In [7]:
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')
# uname
fields
# homedir
# sh

['*', '-2', '-2', 'Unprivileged User']

## 1.3 保留最后N个元素

Q：在迭代操作或者其他操作的时候，怎样只保留最后有限几个元素的历史记录

A：保留有限历史记录正是 ```collections.deque``` 大显身手的时候。比如，下面的代码在多行上面做简单的文本匹配， 并返回匹配所在行的最后N行：

```python
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            yield line, previous_lines
        previous_lines.append(line)

if __name__ == '__main__':
    with open(r'../../cookbook/somefile.txt') as f:   # 这里不存在这个文件
        for line, prevlines in search(f, 'python', 5):
            for pline in prevlines:
                print(pline, end='')
            print(line, end='')
            print('-' * 20)
```

使用 deque(maxlen=N) 构造函数会新建一个固定大小的队列。当新的元素加入并且这个队列已满的时候， 最老的元素会自动被移除掉。

In [23]:
q = deque(maxlen = 3)

In [24]:
q.append(1)
q

deque([1])

In [25]:
q.append(2)
q

deque([1, 2])

In [26]:
q.append(3)
q

deque([1, 2, 3])

In [27]:
q.append(4)
q

deque([2, 3, 4])

在队列两端插入或删除元素时间复杂度都是 ```O(1)``` ，区别于列表，在列表的开头插入或删除元素的时间复杂度为 ```O(N)``` 。

## 1.4 查找最大或最小的N个元素

Q：怎样从一个集合中获得最大或最小的N个元素列表

A：```heapq``` 模块有两个函数：```nlargest()``` 和 ```nsmallest()``` 可以完美解决这个问题。

In [30]:
import heapq
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3, nums))
print(heapq.nsmallest(3, nums))

[42, 37, 23]
[-4, 1, 2]


两个函数都能接受一个关键字参数，用于更复杂的数据结构中：

In [32]:
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price': 91.1},
    {'name': 'AAPL', 'shares': 50, 'price': 543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares': 45, 'price': 16.35},
    {'name': 'ACME', 'shares': 75, 'price': 115.65}
]
cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])
cheap

[{'name': 'YHOO', 'shares': 45, 'price': 16.35},
 {'name': 'FB', 'shares': 200, 'price': 21.09},
 {'name': 'HPQ', 'shares': 35, 'price': 31.75}]

如果你想在一个集合中查找最小或最大的 N 个元素，并且 N 小于集合元素数量，那么这些函数提供了很好的性能。 因为在底层实现里面，首先会先将集合数据进行堆排序后放入一个列表中：

In [33]:
nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
heap = list(nums)
heapq.heapify(heap)  # heapq.heapify 将list转化为heap
heap

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

堆数据结构最重要的特征是 ```heap[0]``` 永远是最小的元素。并且剩余的元素可以很容易的通过调用 ```heapq.heappop()``` 方法得到， 该方法会先将第一个元素弹出来，然后用下一个最小的元素来取代被弹出元素（这种操作时间复杂度仅仅是 O(log N)，N 是堆大小）。 比如，如果想要查找最小的 3 个元素，你可以这样做：

In [34]:
heapq.heappop(heap)

-4

In [36]:
heapq.heappop(heap)

1

当要查找的元素个数相对比较小的时候，函数 ```nlargest()``` 和 ```nsmallest()``` 是很合适的。 如果你仅仅想查找唯一的最小或最大（N=1）的元素的话，那么使用 ```min()``` 和 ```max()``` 函数会更快些。 类似的，如果 N 的大小和集合大小接近的时候，通常先排序这个集合然后再使用切片操作会更快点 （ ```sorted(items)[:N]``` 或者是 ```sorted(items)[-N:]``` ）。 需要在正确场合使用函数 ```nlargest()``` 和 ```nsmallest()``` 才能发挥它们的优势 （如果 N 快接近集合大小了，那么使用排序操作会更好些）。

## 1.5 实现一个优先级队列

Q：怎样实现一个按优先级排序的队列？并且在这个队列上面每次pop操作总是返回优先级最高的元素

A：利用heapq模块实现

In [44]:
import heapq

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
    def pop(self):
        return heapq.heappop(self._queue)[-1]

其使用方式如下:

In [45]:
class Item:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)

In [47]:
q = PriorityQueue()

In [48]:
q.push(Item('foo'), 1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)
q.pop()

Item('bar')

In [49]:
q.pop()

Item('spam')

仔细观察可以发现，第一个 pop() 操作返回优先级最高的元素。 另外注意到如果两个有着相同优先级的元素（ foo 和 grok ），pop 操作按照它们被插入到队列的顺序返回的。